In [151]:
import pyspark
import pyspark.sql.functions as sf
from pyspark.sql import Window
from typing import Literal
import os
import pandas as pd

In [26]:
data_dir = 'data/Tweets/'
master = 'local[2]'

spark = (
    pyspark.sql.SparkSession.builder
    .master(master) 
    .getOrCreate()
)
spark

In [197]:
def load_json_data(dir: str) -> pyspark.sql.dataframe.DataFrame:
    data: pyspark.sql.dataframe.DataFrame = None
    file_list = []
    for file in os.listdir(dir):
        file_list.append(os.path.join(dir, file))
    return spark.read.json(file_list)
        
nested_df = load_json_data(data_dir)

In [198]:
def filter_cols(spark_df: pyspark.sql.dataframe.DataFrame, 
                          columns: list) -> pyspark.sql.dataframe.DataFrame:
    return (
        spark_df
        .select(columns)
    )

filtered_df = filter_cols(nested_df, ["created_at", "entities.hashtags.text"])
filtered_df.limit(10).toPandas()

,created_at,text
0,Thu Feb 18 14:49:30 +0000 2021,[]
1,Thu Feb 18 14:49:30 +0000 2021,[]
2,Thu Feb 18 14:49:30 +0000 2021,[]
3,Thu Feb 18 14:49:30 +0000 2021,[]
4,Thu Feb 18 14:49:30 +0000 2021,[]
5,Thu Feb 18 14:49:30 +0000 2021,[]
6,Thu Feb 18 14:49:30 +0000 2021,[]
7,Thu Feb 18 14:49:30 +0000 2021,[]
8,Thu Feb 18 14:49:30 +0000 2021,[]
9,Thu Feb 18 14:49:30 +0000 2021,"[missingperson, helpfindrat]"


In [199]:
def rename_cols(spark_df: pyspark.sql.dataframe.DataFrame,
                mapping:dict) -> pyspark.sql.dataframe.DataFrame:
    for old_name, new_name in mapping.items():
        spark_df = spark_df.withColumnRenamed(old_name, new_name)
    return spark_df

renamed_df = rename_cols(filtered_df,
                         {'text': 'hashtags'})
renamed_df.limit(5).toPandas()


,created_at,hashtags
0,Thu Feb 18 14:49:30 +0000 2021,[]
1,Thu Feb 18 14:49:30 +0000 2021,[]
2,Thu Feb 18 14:49:30 +0000 2021,[]
3,Thu Feb 18 14:49:30 +0000 2021,[]
4,Thu Feb 18 14:49:30 +0000 2021,[]


In [200]:
def convert_to_datecol(spark_df: pyspark.sql.dataframe.DataFrame,
                   col: str,
                   format: str) -> pyspark.sql.dataframe.DataFrame:
    spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
    df_datecol_converted = (
        spark_df
        .withColumn(col,
                    sf.to_timestamp(col,
                               format=format)
        )
    )
    return df_datecol_converted

preproccesed_df = convert_to_datecol(renamed_df, 'created_at', 'EEE MMM dd HH:mm:ss ZZZZZ yyyy')
preproccesed_df.limit(10).toPandas()
    

,created_at,hashtags
0,2021-02-18 15:49:30,[]
1,2021-02-18 15:49:30,[]
2,2021-02-18 15:49:30,[]
3,2021-02-18 15:49:30,[]
4,2021-02-18 15:49:30,[]
5,2021-02-18 15:49:30,[]
6,2021-02-18 15:49:30,[]
7,2021-02-18 15:49:30,[]
8,2021-02-18 15:49:30,[]
9,2021-02-18 15:49:30,"[missingperson, helpfindrat]"


In [207]:
rolling_window = Window.partitionBy('hashtag').orderBy('hour')
rolling_window_2 = Window.partitionBy('hour')


def truncate_date(spark_df,
                  time: str
                  col: str)

# def explode_col_and_count(spark_df,
#                           col,
#                           )

(
    preproccesed_df
    .withColumn('hour', sf.date_trunc('hour', sf.col('created_at')))
    .withColumn('hashtag', sf.explode(sf.col('hashtags')))
    .groupBy('hour', 'hashtag') 
    .count()
    .withColumn('rel_change',
                (sf.col('count') - sf.lag('count',1).over(rolling_window))/sf.lag('count',1).over(rolling_window))
    
    .withColumn('rel_change',
               sf.when(sf.col('count') < 50, sf.lit(None))
               .otherwise(sf.col('rel_change')))
    
     .withColumn('rel_change',
               sf.when(sf.lag('count',1).over(rolling_window) < 5, sf.lit(None))
               .otherwise(sf.col('rel_change')))

    .withColumn('max_rel_change',
                sf.max('rel_change').over(rolling_window_2))
    .filter(
        sf.col('rel_change') == sf.col('max_rel_change')
    )
    .sort(['hour'], ascending=[True]) 
).toPandas()

,hour,hashtag,count,rel_change,max_rel_change
0,2020-09-17 16:00:00,UniteForIOKandPalestine,120,4.217391,4.217391
1,2020-09-17 17:00:00,이달의소녀,178,24.428571,24.428571
2,2020-09-18 09:00:00,Valimai,100,1.857143,1.857143
3,2020-09-18 10:00:00,काला_कानून_संविदा_वापस_लो,67,10.166667,10.166667
4,2020-09-18 11:00:00,TREASURE_사랑해_ILoveYou,166,17.444444,17.444444
5,2020-09-18 13:00:00,ILANDTheFinale,272,44.333333,44.333333
6,2020-09-18 14:00:00,AOTD41,68,12.600000,12.600000
7,2020-09-18 15:00:00,Dynamite,219,5.083333,5.083333
8,2020-09-18 16:00:00,BTS,1831,15.061404,15.061404
9,2021-02-18 16:00:00,NCT127,872,61.285714,61.285714


In [43]:
nested_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

In [71]:
type((
    nested_df
    .select(["created_at", "text"])
    
    # .select("extended_tweet")
    # .filter(
    #     sf.col("extended_tweet") == sf.col("extended_tweet")
    # )
).limit(10))

pyspark.sql.dataframe.DataFrame